In [3]:
import importlib
import os
import tensorflow as tf
import numpy as np
import cv2
from models.condGAN256 import condGAN256
from models.cycleGAN256 import cycleGAN256

import importlib

checkpoint_path = "./training_checkpoints/ckpt-2"
cGAN = condGAN256()
cGAN.built = True
cGAN.restore_from_checkpoint(checkpoint_path)

def createDir(dir):
    if not os.path.exists(dir):
        os.mkdir(dir)

def normalize(input_image, real_image):
  input_image = (input_image / 127.5) - 1
  real_image = (real_image / 127.5) - 1

  return input_image, real_image

def load_double(image_file):

  image = cv2.imread(image_file, cv2.IMREAD_COLOR)
  _, w, _= np.shape(image)
  w = w // 2

  he = image[:, :w, :]
  mt = image[:, w:, :]

  return he, mt

def load_image(image_file):
  he, mt  = load_double(image_file)

  return he, mt

def TF2CV(im):
    img = tf.cast(tf.math.scalar_mul(255/2, im[0]+1), dtype=tf.uint8)
    img_ = np.array(tf.keras.utils.array_to_img(img),dtype='uint8')
    img_ = cv2.cvtColor(img_, cv2.COLOR_RGB2BGR)
    return img_

def histEqu(img):
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)

    # equalize the histogram of the Y channel
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])

    # convert the YUV image back to RGB format
    img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

    return img_output

sample_image_path = "./ch0_0_1536.png"

he, mt = load_image(sample_image_path)
he_, mt_ = normalize(he, mt)
he_ = np.expand_dims(he_, axis=0)

mt_virtual = cGAN(he_)

mt_virtual_ = TF2CV(mt_virtual)

out = cv2.hconcat((he, mt, mt_virtual_))

cv2.imwrite(f"./predictions/ch0_0_1536_.png",out)



True